## Notebook Setup and Package Installation

In [2]:
! pip install geopandas
! pip install fiona
! pip install imblearn
! pip install basemap

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
# from geopy.geocoders import Nominatim
import geopandas as gpd
import fiona
# from shapely.geometry import Point, Polygon
# import plotly.express as px
# import datetime
# import folium
# from geopy.distance import geodesic
import pathlib
import urllib
import seaborn as sns
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from datetime import datetime


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [5]:
%run Utility_Functions.ipynb

## Read Datasets in

In [6]:
hurricane_exposure = pd.read_csv('../data/ibtracs.NA.csv')
zip_to_fips_mapping = pd.read_csv('../data/ZIP-COUNTY-FIPS_2017-06.csv')
nri_data = pd.read_csv('../data/NRI_Table_Counties.csv')
rent_data_with_location = pd.read_csv('../data/rent_data_with_location.csv')
rent_data = pd.read_csv('../data/final_rent_data.csv')
macro_data = pd.read_csv('../data/macro_data.csv') 

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1,2,3,8,9,14,19,20,23,24,161,162) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Filter Datasets to Get Full Dataset or Limited Dataset [---HERE---]

In [7]:

###########################################
##### For FULL dataset, run this cell #####
###########################################

seasons = list(range(1995, 2022))


In [8]:

##############################################
##### For LIMITED dataset, run this cell #####
##############################################

seasons = list(range(2010, 2019))


## Data Munging for Each Dataset

### Rent Data

##### Get quarter & Subset columns to those we are keeping 

In [9]:
rent_data_with_location['full_address'] = rent_data_with_location['prop_index'] + ', ' + rent_data_with_location['address']
rent_data_with_location_filtered = rent_data_with_location[
    ['street_address', 'LAT', 'LON']
].drop_duplicates().groupby('street_address').first().reset_index()

In [10]:
rent_data_with_location_filtered.head()

,street_address,LAT,LON
0,1 Curtiss Pkwy,25.821677,-80.280869
1,1 Laurel Oaks Dr,28.701085,-81.322750
2,1 S Pine Island Rd,26.120060,-80.265236
3,100 Acklins Cir,29.154999,-81.071671
4,100 Alva Cir,29.225818,-81.087221


In [11]:
rent_data_merged = pd.merge(rent_data, rent_data_with_location_filtered, on='street_address')

In [12]:
rent_data_filtered = rent_data_merged[
    ['address', 'street_address', 'city', 'state', 'zip',
    'occupancy', 'effective_rent', 'prop_rent_growth',
    'sma_rent_growth', 'quarter_x',  'LAT', 'LON', 'rent_decreased'] 
].rename(columns={'city_x': 'city', 'quarter_x': 'quarter_period'})

rent_data_filtered['quarter'] = (
    pd.to_datetime(rent_data_filtered['quarter_period'] , errors='coerce').apply(get_quarter_from_date)
)

get_year = lambda x: x.split('-')[0]
rent_data_filtered['Year'] = rent_data_filtered['quarter_period'].apply(get_year)
rent_data_filtered['Year'] = rent_data_filtered['Year'].astype('int')

rent_data_filtered = rent_data_filtered.dropna(axis=0, how = 'any') ## drop any rows that have at least one NA value

rent_data_filtered['street_address'] = rent_data_filtered['street_address'].str.lower()

rent_data_filtered = rent_data_filtered[
    rent_data_filtered.Year.isin(seasons)
]
rent_data_filtered['quarter_period'] = pd.to_datetime(rent_data_filtered['quarter_period'] ) 

rent_data_filtered['address'] = rent_data_filtered['address'].str.lower()

rent_data_filtered['next_efective_rent'] = rent_data_filtered.groupby(
    ['address']
)['effective_rent'].shift(-1)

rent_data_filtered['rent_decreased'] = np.where(
    rent_data_filtered['next_efective_rent'] < rent_data_filtered['effective_rent'], 1, 0
)



rent_data_filtered.head()

,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent
181,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9700,1412.0,0.010341,0.008450,2012-06-30,25.986308,-80.308144,0,2Q12,2012,1454.0
182,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9631,1454.0,0.010865,0.011736,2012-09-30,25.986308,-80.308144,0,3Q12,2012,1472.0
183,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9447,1472.0,0.010905,0.012355,2012-12-31,25.986308,-80.308144,0,4Q12,2012,1478.0
184,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9516,1478.0,0.010730,0.016852,2013-03-31,25.986308,-80.308144,0,1Q13,2013,1481.0
185,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9608,1481.0,0.010512,0.006933,2013-06-30,25.986308,-80.308144,0,2Q13,2013,1516.0


In [13]:

rent_data_filtered[
    (rent_data_filtered.address == '[27] parc500, 500 n congress ave, west palm beach, fl, 33401') &
    (rent_data_filtered.quarter == '4Q16')
].drop_duplicates()

,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent
156219,"[27] parc500, 500 n congress ave, west palm be...",500 n congress ave,West Palm Beach,FL,33401,0.9032,1193.0,0.007983,0.005005,2016-12-31,26.717162,-80.080033,0,4Q16,2016,1342.0


In [14]:
rent_data_filtered.shape

(74330, 16)

In [15]:
# rent_data_filtered_sorted = rent_data_filtered.sort_values(by=['street_address', 'quarter_period'])

In [16]:
# def calculate_valid_date_range(d):
#     for i in range(len(d) - 1):
#         current, _next = d[i], d[i + 1]
#         diff = _next - current
#         if diff.days > 92:
#             return False
#     return True
    
    

In [17]:
# properties_check = rent_data_filtered_sorted.groupby(['street_address'])['quarter_period'].agg(list).reset_index()

In [18]:
# properties_check['valid_date_range'] = properties_check['quarter_period'].apply(calculate_valid_date_range)

In [19]:
# properties_check.head()

In [20]:
# valid_date_properties = properties_check[properties_check.valid_date_range == True]
# valid_addresses = list(valid_date_properties.street_address)

In [21]:
# rent_data_filtered = rent_data_filtered[
#     rent_data_filtered.street_address.isin(valid_addresses)
# ]

In [22]:
# rent_data_filtered.head()

In [23]:
# # Create Derived Features:
# # Next Effective Rent, 
# # Rent Difference, 
# # Percent Change, 
# # Rent Decreased

# # Update expose_status_code

# rent_data_change_in_rent = rent_data_filtered.sort_values(by=['address', 'Year', 'quarter']).reset_index()

# rent_data_change_in_rent['id'] = rent_data_change_in_rent.groupby(['address']).ngroup()
# rent_data_change_in_rent['Next Effective Rent'] = rent_data_change_in_rent.groupby(
#     ['id']
# )['effective_rent'].shift(-1)

# rent_data_change_in_rent['Rent Difference'] = (
#     rent_data_change_in_rent['Next Effective Rent'] - 
#     rent_data_change_in_rent['effective_rent']
# )
# rent_data_change_in_rent['Percent Change'] = (
#     np.divide(
#         rent_data_change_in_rent['Next Effective Rent'] - exposed_property_data_filtered['effective_rent'],
#         rent_data_change_in_rent['effective_rent']
#     )
# )
# rent_data_change_in_rent['Rent Decreased'] = np.where(
#     rent_data_change_in_rent['Percent Change'] < 0,
#     1, 0
# )

# # # Update expose_status_code, if hurricane_hit == 1, expose_status_code = exposed_property_data_filtered['expose_status_code'] 
# # # otherwise expose_status_code = 0
# # rent_data_change_in_rent['expose_status_code'] = np.where(
# #     rent_data_change_in_rent.hurricane_hit != 1,
# #     0, rent_data_change_in_rent['expose_status_code']
# # )

# # rent_data_change_in_rent = rent_data_change_in_rent.drop_duplicates().dropna()
# rent_data_change_in_rent = rent_data_change_in_rent.drop(columns=['index', 'id'])
# rent_data_change_in_rent = rent_data_change_in_rent.sort_values(by=['address', 'Year', 'quarter']).reset_index()

# rent_data_change_in_rent.to_csv(f'../processed_data/property_data_2010_2019.parquet')
# print(rent_data_change_in_rent.shape)
# rent_data_change_in_rent.head()


In [24]:
rent_data_filtered.to_csv(f'../processed_data/property_data_2010_2019.parquet', index=False)
rent_data_filtered = pd.read_csv(f'../processed_data/property_data_2010_2019.parquet')

In [25]:

rent_data_filtered.head()


,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent
0,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9700,1412.0,0.010341,0.008450,2012-06-30,25.986308,-80.308144,0,2Q12,2012,1454.0
1,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9631,1454.0,0.010865,0.011736,2012-09-30,25.986308,-80.308144,0,3Q12,2012,1472.0
2,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9447,1472.0,0.010905,0.012355,2012-12-31,25.986308,-80.308144,0,4Q12,2012,1478.0
3,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9516,1478.0,0.010730,0.016852,2013-03-31,25.986308,-80.308144,0,1Q13,2013,1481.0
4,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9608,1481.0,0.010512,0.006933,2013-06-30,25.986308,-80.308144,0,2Q13,2013,1516.0


### 'Distance From Coast' Data

In [26]:
address_with_distance_from_coast = rent_data_with_location_filtered

address_with_distance_from_coast['distance_from_cost'] = (
     np.vectorize(distance_from_coast)(
         address_with_distance_from_coast['LON'], 
         address_with_distance_from_coast['LAT']
     )
)

address_with_distance_from_coast['street_address'] = address_with_distance_from_coast['street_address'].str.lower()

address_with_distance_from_coast = address_with_distance_from_coast.drop_duplicates().dropna().drop(columns=['LON', 'LAT'])
address_with_distance_from_coast.head()

,street_address,distance_from_cost
0,1 curtiss pkwy,17.784949
1,1 laurel oaks dr,56.202363
2,1 s pine island rd,25.978704
3,100 acklins cir,16.717810
4,100 alva cir,22.867683


### ZIP & NRI Data

In [27]:
zip_to_fips_mapping = zip_to_fips_mapping[
    ['ZIP', 'STCOUNTYFP']
]
nri_data_filtered = nri_data[
    ['NRI_ID', 'STATE', 'STATEABBRV', 'COUNTY', 'STCOFIPS', 
     'RISK_SCORE', 'CFLD_RISKS', 'HRCN_RISKS']
]
nri_data_filtered = nri_data_filtered[nri_data_filtered.STATEABBRV == 'FL']

nri_data_to_zip = pd.merge(nri_data_filtered, zip_to_fips_mapping, left_on='STCOFIPS', right_on='STCOUNTYFP')
nri_data_to_zip = nri_data_to_zip.groupby('ZIP').agg({'RISK_SCORE': 'max', 'CFLD_RISKS': 'max', 'HRCN_RISKS': 'max'}).reset_index()
nri_data_to_zip.head()

,ZIP,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
0,32003,12.343169,13.322964,22.572425
1,32004,12.821337,15.630605,23.585659
2,32006,12.343169,13.322964,22.572425
3,32007,16.088802,17.234311,28.678636
4,32008,8.824387,9.270481,14.725039


In [28]:
nri_data_to_zip.shape

(1401, 4)

### Hurricane Data

In [29]:
hurricane_data = hurricane_exposure[hurricane_exposure.SEASON.isin(seasons)][
    (hurricane_exposure.USA_STATUS == 'HU') |
    (hurricane_exposure.USA_STATUS == 'HR')
][
    (hurricane_exposure.NATURE == 'TS')
]

hurricane_df = pd.DataFrame(hurricane_data, columns=[
    'SID', 'NAME', 'SEASON', 'BASIN', 'LAT', 'LON', 'USA_ROCI', 'USA_RMW', 'ISO_TIME', 'BOM_EYE', 'USA_WIND', 'USA_GUST',
    'USA_SEAHGT', 'USA_SSHS'
])
hurricane_df.drop(hurricane_df.index[0], inplace=True)
hurricane_df["ISO_TIME"] = pd.to_datetime(hurricane_df["ISO_TIME"], errors='coerce')
hurricane_df["MONTH"] = hurricane_df["ISO_TIME"].dt.month
hurricane_df['USA_RMW'] = pd.to_numeric(hurricane_df['USA_RMW'], errors='coerce')
hurricane_df['USA_ROCI'] = pd.to_numeric(hurricane_df['USA_ROCI'], errors='coerce')
hurricane_df['USA_GUST'] = pd.to_numeric(hurricane_df['USA_GUST'], errors='coerce')
hurricane_df['USA_SEAHGT'] = pd.to_numeric(hurricane_df['USA_SEAHGT'], errors='coerce')
hurricane_df['RADIUS_KM'] = hurricane_df['USA_RMW'].apply(lambda x: float(x) * 1.852)
hurricane_df['LON'] = pd.to_numeric(hurricane_df['LON'], errors='coerce')
hurricane_df['LAT'] = pd.to_numeric(hurricane_df['LAT'], errors='coerce')
hurricane_df.dropna(subset=['LAT', 'LON'], inplace=True)

hurricane_df['Quarters_hurricane'] = np.vectorize(get_quarter_from_month)(hurricane_df['MONTH'], hurricane_df['SEASON'])

filtered_hurricane_tracks = hurricane_df.reset_index(drop=True)
filtered_hurricane_tracks = filtered_hurricane_tracks[
    ['SID', 'SEASON', 'BASIN', 'NAME', 'MONTH', 'LAT', 'LON', 'RADIUS_KM', 'ISO_TIME', 'Quarters_hurricane', 
     'USA_ROCI', 'USA_RMW', 'BOM_EYE', 'USA_WIND', 'USA_GUST',
    'USA_SEAHGT', 'USA_SSHS']
].rename(
    columns={
        'LON': 'HUR_LON',
        'LAT':'HUR_LAT'
    }
)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [30]:
filtered_hurricane_tracks.head()

,SID,SEASON,BASIN,NAME,MONTH,HUR_LAT,HUR_LON,RADIUS_KM,ISO_TIME,Quarters_hurricane,USA_ROCI,USA_RMW,BOM_EYE,USA_WIND,USA_GUST,USA_SEAHGT,USA_SSHS
0,2010176N16278,2010,NaN,ALEX,6,23.0273,-94.6301,27.78,2010-06-30 03:00:00,2Q10,250.0,15.0,,67,NaN,12.0,1
1,2010176N16278,2010,NaN,ALEX,6,23.1000,-94.8000,27.78,2010-06-30 06:00:00,2Q10,250.0,15.0,,70,85.0,12.0,1
2,2010176N16278,2010,NaN,ALEX,6,23.2181,-94.9798,27.78,2010-06-30 09:00:00,2Q10,250.0,15.0,,72,NaN,12.0,1
3,2010176N16278,2010,NaN,ALEX,6,23.5000,-95.2000,27.78,2010-06-30 12:00:00,2Q10,250.0,15.0,,75,85.0,12.0,1
4,2010176N16278,2010,NaN,ALEX,6,24.0000,-95.5000,27.78,2010-06-30 15:00:00,2Q10,250.0,15.0,,75,NaN,12.0,1


### Macro Data

In [31]:
# macro_data_filtered = macro_data[
#     ['street_address', 'quarter_x', 'fed_funds',
#        '10_yr_inflation', '5_yr_inflation', 'mortgage', 'delinquency',
#        'housing_inventory', 'bank_tightening']
# ].rename(columns={'quarter_x': 'quarter_period'})

# macro_data_filtered['quarter'] = (
#     pd.to_datetime(macro_data_filtered['quarter_period'] , errors='coerce').apply(get_quarter_from_date)
# )


# # macro_data_filtered['address'] = np.vectorize(build_address)(
# #     macro_data_filtered['street'], macro_data_filtered['city'],
# #     macro_data_filtered['state'], macro_data_filtered['zip']
# # )

# # # TODO: Something wrong with Macro Data, how Effective Rent & Next Q Rent and y being generated?

# # # Removed Effective Rent since its redundant and gave us different result from rent data
# # macro_data_filtered = macro_data_filtered.drop(columns=['Street', 'City', 'State', 'Zip', 'Quarter', 'Effective Rent', 'Next Q Rent', 'y']) # Removed Quarter since its always 2017-09-30

# # macro_data_filtered['address'] = macro_data_filtered['address'].str.lower()
# # macro_data_filtered.head()

# macro_data.head()

In [32]:
#macro_data_filtered['address'].nunique()

## Joining Datasets (Rent & CoastDistance & NRI & Macro)

##############################################

Rent Data: rent_data_filtered

Distance from Coast: address_with_distance_from_coast

NRI Data: nri_data_to_zip

Hurricane Data: filtered_hurricane_tracks

Macro Data: macro_data_filtered

##############################################

### Joining all data (except Hurricane) together: Rent Data +  Distance from Coast + NRI + Macro

### 1. Join Rent Data with Distance from Coast on address

In [33]:
# 1 new column: distance_from_cost
rent_data_filtered['street_address'] = rent_data_filtered['street_address'].str.lower()
Rent_Data_with_CoastDistance = rent_data_filtered.merge(address_with_distance_from_coast, on='street_address', how='left')
Rent_Data_with_CoastDistance.head()

,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent,distance_from_cost
0,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9700,1412.0,0.010341,0.008450,2012-06-30,25.986308,-80.308144,0,2Q12,2012,1454.0,20.70975
1,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9631,1454.0,0.010865,0.011736,2012-09-30,25.986308,-80.308144,0,3Q12,2012,1472.0,20.70975
2,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9447,1472.0,0.010905,0.012355,2012-12-31,25.986308,-80.308144,0,4Q12,2012,1478.0,20.70975
3,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9516,1478.0,0.010730,0.016852,2013-03-31,25.986308,-80.308144,0,1Q13,2013,1481.0,20.70975
4,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9608,1481.0,0.010512,0.006933,2013-06-30,25.986308,-80.308144,0,2Q13,2013,1516.0,20.70975


In [34]:
rent_data_filtered[
    (rent_data_filtered.address == '[27] parc500, 500 n congress ave, west palm beach, fl, 33401') &
    (rent_data_filtered.quarter == '4Q16')
]

,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent
33515,"[27] parc500, 500 n congress ave, west palm be...",500 n congress ave,West Palm Beach,FL,33401,0.9032,1193.0,0.007983,0.005005,2016-12-31,26.717162,-80.080033,0,4Q16,2016,1342.0


### 2. Join Rent_Data_with_CoastDistance with NRI Data on Zip Code

In [35]:
# 3 new columns: RISK_SCORE, CFLD_RISKS, HRCN_RISKS
Rent_Data_with_Coast_NRI = pd.merge(Rent_Data_with_CoastDistance, nri_data_to_zip, left_on='zip', right_on='ZIP')
Rent_Data_with_Coast_NRI = Rent_Data_with_Coast_NRI.drop(
    columns=['ZIP']
)
print(Rent_Data_with_Coast_NRI.shape)
Rent_Data_with_Coast_NRI.head()

(74330, 20)


,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
0,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9700,1412.0,0.010341,0.008450,2012-06-30,25.986308,-80.308144,0,2Q12,2012,1454.0,20.70975,36.983895,20.000711,67.03555
1,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9631,1454.0,0.010865,0.011736,2012-09-30,25.986308,-80.308144,0,3Q12,2012,1472.0,20.70975,36.983895,20.000711,67.03555
2,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9447,1472.0,0.010905,0.012355,2012-12-31,25.986308,-80.308144,0,4Q12,2012,1478.0,20.70975,36.983895,20.000711,67.03555
3,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9516,1478.0,0.010730,0.016852,2013-03-31,25.986308,-80.308144,0,1Q13,2013,1481.0,20.70975,36.983895,20.000711,67.03555
4,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9608,1481.0,0.010512,0.006933,2013-06-30,25.986308,-80.308144,0,2Q13,2013,1516.0,20.70975,36.983895,20.000711,67.03555


### 3. Join Rent_Data_with_Coast_NRI with Macro Data on address

In [36]:
# # 8 new columns: SMA rent growth	fed_funds	10_yr_inflation	5_yr_inflation	mortgage	delinquency	housing_inventory	bank_tightening

# Rent_Data_with_Coast_NRI_Macro = Rent_Data_with_Coast_NRI.merge(macro_data_filtered, on='address', how='left')
# Rent_Data_with_Coast_NRI_Macro.head()

## Join Rent Data (Raw, Not joined with anything) with Hurricane Data

In [37]:
#self join table
property_to_hurricane_tracks = pd.merge(
    rent_data_filtered.assign(key=1),
    filtered_hurricane_tracks.assign(key=1), 
    on='key', 
    suffixes=('', '_2')
).drop('key', axis=1)

#find distances
property_to_hurricane_tracks['dist'] = haversine(
    property_to_hurricane_tracks['LON'], property_to_hurricane_tracks['LAT'], 
    property_to_hurricane_tracks['HUR_LON'], property_to_hurricane_tracks['HUR_LAT'])


In [38]:
rent_data_filtered.head()

,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,LAT,LON,rent_decreased,quarter,Year,next_efective_rent
0,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9700,1412.0,0.010341,0.008450,2012-06-30,25.986308,-80.308144,0,2Q12,2012,1454.0
1,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9631,1454.0,0.010865,0.011736,2012-09-30,25.986308,-80.308144,0,3Q12,2012,1472.0
2,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9447,1472.0,0.010905,0.012355,2012-12-31,25.986308,-80.308144,0,4Q12,2012,1478.0
3,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9516,1478.0,0.010730,0.016852,2013-03-31,25.986308,-80.308144,0,1Q13,2013,1481.0
4,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.9608,1481.0,0.010512,0.006933,2013-06-30,25.986308,-80.308144,0,2Q13,2013,1516.0


In [39]:
property_to_hurricane_tracks['expose_status'] = np.vectorize(is_exposed)(property_to_hurricane_tracks['dist'], property_to_hurricane_tracks['RADIUS_KM'])

In [40]:
property_to_hurricane_tracks_exposed = property_to_hurricane_tracks[
    property_to_hurricane_tracks.expose_status != 'No Exposure'
]

In [41]:
property_to_hurricane_tracks_exposed.shape

(170275, 35)

In [42]:
# Create Hurricane_hit variable
property_to_hurricane_tracks_exposed['hurricane_hit'] = np.where(
    property_to_hurricane_tracks_exposed['quarter'] == property_to_hurricane_tracks_exposed['Quarters_hurricane'],
    1, 0
)

In [43]:
property_to_hurricane_tracks_exposed = property_to_hurricane_tracks_exposed.drop(
    columns=['LON', 'LAT', 'HUR_LAT', 'HUR_LON', 'Quarters_hurricane']
).drop_duplicates()

property_to_hurricane_tracks_exposed.head()


,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,quarter,Year,next_efective_rent,SID,SEASON,BASIN,NAME,MONTH,RADIUS_KM,ISO_TIME,USA_ROCI,USA_RMW,BOM_EYE,USA_WIND,USA_GUST,USA_SEAHGT,USA_SSHS,dist,expose_status,hurricane_hit
103659,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9259,661.0,0.003012,0.002202,2010-03-31,0,1Q10,2010,691.0,2016273N13300,2016,NaN,MATTHEW,10,37.04,2016-10-07 12:00:00,270.0,20.0,,105,130.0,12.0,3,108.384380,Low Exposure,0
103660,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9259,661.0,0.003012,0.002202,2010-03-31,0,1Q10,2010,691.0,2016273N13300,2016,NaN,MATTHEW,10,37.04,2016-10-07 15:00:00,270.0,20.0,,102,NaN,12.0,3,116.716455,Low Exposure,0
103877,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9259,661.0,0.003012,0.002202,2010-03-31,0,1Q10,2010,691.0,2017242N16333,2017,NaN,IRMA,9,37.04,2017-09-11 06:00:00,350.0,20.0,,65,90.0,12.0,1,92.915628,Low Exposure,0
103878,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9259,661.0,0.003012,0.002202,2010-03-31,0,1Q10,2010,691.0,2017242N16333,2017,NaN,IRMA,9,55.56,2017-09-11 09:00:00,350.0,30.0,,57,NaN,12.0,0,117.267323,Low Exposure,0
105557,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9240,691.0,0.003730,0.002662,2010-06-30,0,2Q10,2010,706.0,2016273N13300,2016,NaN,MATTHEW,10,37.04,2016-10-07 12:00:00,270.0,20.0,,105,130.0,12.0,3,108.384380,Low Exposure,0


In [44]:
# Create Expose Statue Code
property_to_hurricane_tracks_exposed['expose_status_code'] = np.where(
    property_to_hurricane_tracks_exposed['expose_status'] == 'High Exposure', 3,
    np.where(
        property_to_hurricane_tracks_exposed['expose_status'] == 'Risk', 2,
        np.where(
            property_to_hurricane_tracks_exposed['expose_status'] == 'Low Exposure', 1,
            0
        )
    )
)

## Checkpoint 1 - Rent & Hurricane Joined [--- UPDATE FILE NAME---]

In [45]:
###########################################
######### Update the file name  ###########
###########################################

property_to_hurricane_tracks_exposed.to_parquet('../processed_data/LIMITED_property_hurricane_2010_2019_NEW1.parquet')

In [ ]:
###########################################
######### Update the file name  ###########
###########################################

property_to_hurricane_tracks_exposed = pd.read_parquet('../processed_data/LIMITED_property_hurricane_2010_2019_NEW1.parquet')
print(property_to_hurricane_tracks_exposed.shape)
property_to_hurricane_tracks_exposed.head()

In [46]:
set(property_to_hurricane_tracks_exposed.expose_status_code)

{1, 2, 3}

## Get Only Exposed Properties & Create Derived Features

In [47]:
# Get only hurricane_hit == 1 and expose_status_code != 'No Exposure'
hit_addresses = property_to_hurricane_tracks_exposed[
    (property_to_hurricane_tracks_exposed.hurricane_hit == 1) &
    (property_to_hurricane_tracks_exposed.expose_status_code.isin([1,2,3]))
]['street_address']

In [48]:
# Get all rows for properties that have ever been hit for all years

hit_properties_full_data = property_to_hurricane_tracks_exposed[
    property_to_hurricane_tracks_exposed['street_address'].isin(hit_addresses)
]
hit_properties_full_data = hit_properties_full_data[
    hit_properties_full_data.hurricane_hit == 1
]
print(hit_properties_full_data.shape)
hit_properties_full_data.head()

(5496, 32)


,address,street_address,city,state,zip,occupancy,effective_rent,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,quarter,Year,next_efective_rent,SID,SEASON,BASIN,NAME,MONTH,RADIUS_KM,ISO_TIME,USA_ROCI,USA_RMW,BOM_EYE,USA_WIND,USA_GUST,USA_SEAHGT,USA_SSHS,dist,expose_status,hurricane_hit,expose_status_code
154905,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9297,991.0,0.007375,0.006071,2016-12-31,0,4Q16,2016,1088.0,2016273N13300,2016,NaN,MATTHEW,10,37.040,2016-10-07 12:00:00,270.0,20.0,,105,130.0,12.0,3,108.384380,Low Exposure,1,1
154906,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9297,991.0,0.007375,0.006071,2016-12-31,0,4Q16,2016,1088.0,2016273N13300,2016,NaN,MATTHEW,10,37.040,2016-10-07 15:00:00,270.0,20.0,,102,NaN,12.0,3,116.716455,Low Exposure,1,1
160817,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9525,1080.0,0.008197,0.008267,2017-09-30,1,3Q17,2017,1048.0,2017242N16333,2017,NaN,IRMA,9,37.040,2017-09-11 06:00:00,350.0,20.0,,65,90.0,12.0,1,92.915628,Low Exposure,1,1
160818,"[10] barber park, 5300 lake margaret dr, orlan...",5300 lake margaret dr,Orlando,FL,32812,0.9525,1080.0,0.008197,0.008267,2017-09-30,1,3Q17,2017,1048.0,2017242N16333,2017,NaN,IRMA,9,55.560,2017-09-11 09:00:00,350.0,30.0,,57,NaN,12.0,0,117.267323,Low Exposure,1,1
181694,"[10] bayshore and venture rentals, 4104 20th s...",4104 20th st w,Bradenton,FL,34205,1.0000,655.0,0.080248,0.010642,2017-09-30,1,3Q17,2017,647.0,2017242N16333,2017,NaN,IRMA,9,31.484,2017-09-11 03:00:00,350.0,17.0,,72,NaN,12.0,1,66.731674,Low Exposure,1,1


### Group by property & Year-Quarter
#### Get mean for effective_rent, get the highest expose status code, get the highest hurricane_hit for each group

In [49]:
group_by_columns = ['quarter', 'Year', 'address']

In [50]:
# Group by property & Year-Quarter, 
# Get mean for effective_rent, get the highest expose status code, get the highest hurricane_hit

exposed_property_data_filtered = hit_properties_full_data.groupby(group_by_columns).agg(
    {
        'USA_WIND': 'mean', 
        'USA_GUST': 'mean',
        'USA_SSHS': 'mean',
        'effective_rent': 'mean',
        'expose_status_code': 'max',
        'hurricane_hit': 'max'
    }
)

# Drop Duplicates
exposed_property_data_filtered = exposed_property_data_filtered.drop_duplicates().reset_index()
print(exposed_property_data_filtered.shape)
exposed_property_data_filtered.tail()

(2070, 9)


,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit
2065,4Q18,2018,"[8] mission west, 2651 vista rise, tallahassee...",121.0,165.0,3.500000,620.0,1,1
2066,4Q18,2018,"[9] brookwood, 410 victory garden dr, tallahas...",107.0,NaN,3.000000,727.0,1,1
2067,4Q18,2018,"[9] cottages by the bay, 6200 cypress point dr...",136.0,165.0,4.333333,750.0,2,1
2068,4Q18,2018,"[9] georgetown manor, 524 w tharpe st, tallaha...",107.0,NaN,3.000000,845.0,1,1
2069,4Q18,2018,"[9] oakwood reserve, 2616 mission rd, tallahas...",121.0,165.0,3.500000,644.0,1,1


In [51]:
set(exposed_property_data_filtered.expose_status_code)

{1, 2, 3}

In [52]:
exposed_property_data_filtered[
    exposed_property_data_filtered.address == '[27] parc500, 500 n congress ave, west palm beach, fl, 33401'
]

,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit
1540,4Q16,2016,"[27] parc500, 500 n congress ave, west palm be...",112.333333,135.0,3.333333,1193.0,1,1


In [53]:
exposed_property_data_filtered.head()

,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit
0,3Q16,2016,"[10] grayson place, 1600 pullen rd, tallahasse...",68.333333,80.0,1.0,733.0,3,1
1,3Q16,2016,"[10] sabal court, 2125 jackson bluff rd, talla...",68.333333,80.0,1.0,655.0,3,1
2,3Q16,2016,"[11] capital place at southwood, 2300 bluff oa...",68.333333,80.0,1.0,1233.0,3,1
3,3Q16,2016,"[11] live oaks at 275, 275 john knox rd, talla...",68.333333,80.0,1.0,786.0,3,1
4,3Q16,2016,"[12] capital ridge, 3255 capital cir ne, talla...",68.333333,80.0,1.0,707.0,3,1


### Create Derived Features 
#### Next Effective Rent, Rent Difference, Percent Change, Rent Decreased

In [54]:
# # Create Derived Features:
# # Next Effective Rent, 
# # Rent Difference, 
# # Percent Change, 
# # Rent Decreased

# # Update expose_status_code

# exposed_property_data_filtered = exposed_property_data_filtered.sort_values(by=['address', 'Year', 'quarter']).reset_index()

# exposed_property_data_filtered['id'] = exposed_property_data_filtered.groupby(['address']).ngroup()
# exposed_property_data_filtered['Next Effective Rent'] = exposed_property_data_filtered.groupby(
#     ['id']
# )['effective_rent'].shift(-1)

# exposed_property_data_filtered.head()
# exposed_property_data_filtered['Rent Difference'] = (
#     exposed_property_data_filtered['Next Effective Rent'] - 
#     exposed_property_data_filtered['effective_rent']
# )
# exposed_property_data_filtered['Percent Change'] = (
#     np.divide(
#         exposed_property_data_filtered['Next Effective Rent'] - exposed_property_data_filtered['effective_rent'],
#         exposed_property_data_filtered['effective_rent']
#     )
# )
# exposed_property_data_filtered['Rent Decreased'] = np.where(
#     exposed_property_data_filtered['Percent Change'] < 0,
#     1, 0
# )

# # Update expose_status_code, if hurricane_hit == 1, expose_status_code = exposed_property_data_filtered['expose_status_code'] 
# # otherwise expose_status_code = 0
# exposed_property_data_filtered['expose_status_code'] = np.where(
#     exposed_property_data_filtered.hurricane_hit != 1,
#     0, exposed_property_data_filtered['expose_status_code']
# )

# exposed_property_data_filtered = exposed_property_data_filtered.drop_duplicates().dropna()
# exposed_property_data_filtered = exposed_property_data_filtered.drop(columns=['index', 'id'])
# exposed_property_data_filtered.head()

## Join Exposed Properties(with Derived Features) with Full Joined Dataset (Rent_Data_with_Coast_NRI_Macro)

############################################################################################

Full Joined Data(Rent Data + Distance from Coast + NRI + Macro): Rent_Data_with_Coast_NRI_Macro

Property Exposed Data (with Derived Features):  exposed_property_data_filtered

############################################################################################

In [55]:
# # Removed duplicated and used columns from Rent_Data_with_Coast_NRI_Macro, get ready for the join

# Rent_Data_with_Coast_NRI_Macro_cleaned = Rent_Data_with_Coast_NRI_Macro.drop(
#     columns=['effective_rent', 'LAT', 'LON', 'Year', 'street_address']
# )
# Rent_Data_with_Coast_NRI_Macro_cleaned.head()

Rent_Data_with_Coast_NRI_Macro_cleaned = Rent_Data_with_Coast_NRI.drop(
    columns=['effective_rent', 'LAT', 'LON', 'Year']
)


Rent_Data_with_Coast_NRI_Macro_cleaned.head(1)

# set(Rent_Data_with_Coast_NRI_Macro_cleaned.distance_from_cost)

,address,street_address,city,state,zip,occupancy,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,quarter,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
0,"[10] aventine at miramar ii, 2436 centergate d...",2436 centergate dr,Miramar,FL,33025,0.97,0.010341,0.00845,2012-06-30,0,2Q12,1454.0,20.70975,36.983895,20.000711,67.03555


In [56]:
Rent_Data_with_Coast_NRI_Macro_cleaned[
    (Rent_Data_with_Coast_NRI_Macro_cleaned.address == '[27] parc500, 500 n congress ave, west palm beach, fl, 33401') &
    (Rent_Data_with_Coast_NRI_Macro_cleaned.quarter == '4Q16')
]

,address,street_address,city,state,zip,occupancy,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,quarter,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
6706,"[27] parc500, 500 n congress ave, west palm be...",500 n congress ave,West Palm Beach,FL,33401,0.9032,0.007983,0.005005,2016-12-31,0,4Q16,1342.0,11.2338,38.927992,28.332581,70.963202


In [57]:
full_data = exposed_property_data_filtered.merge(Rent_Data_with_Coast_NRI_Macro_cleaned, on=['address', 'quarter'], how='inner')

full_data.head()

,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit,street_address,city,state,zip,occupancy,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
0,3Q16,2016,"[10] grayson place, 1600 pullen rd, tallahasse...",68.333333,80.0,1.0,733.0,3,1,1600 pullen rd,Tallahassee,FL,32303,0.8929,0.006711,0.005882,2016-09-30,0,748.0,39.482005,11.908661,2.717984,13.908509
1,3Q16,2016,"[10] sabal court, 2125 jackson bluff rd, talla...",68.333333,80.0,1.0,655.0,3,1,2125 jackson bluff rd,Tallahassee,FL,32304,0.9453,0.003298,0.005882,2016-09-30,0,690.0,34.844183,11.908661,2.717984,13.908509
2,3Q16,2016,"[11] capital place at southwood, 2300 bluff oa...",68.333333,80.0,1.0,1233.0,3,1,2300 bluff oak way,Tallahassee,FL,32311,0.9444,0.002880,0.005882,2016-09-30,0,1241.0,30.685704,11.908661,2.717984,13.908509
3,3Q16,2016,"[11] live oaks at 275, 275 john knox rd, talla...",68.333333,80.0,1.0,786.0,3,1,275 john knox rd,Tallahassee,FL,32303,0.9009,0.000797,0.005882,2016-09-30,1,773.0,37.904028,11.908661,2.717984,13.908509
4,3Q16,2016,"[12] capital ridge, 3255 capital cir ne, talla...",68.333333,80.0,1.0,707.0,3,1,3255 capital cir ne,Tallahassee,FL,32308,0.8889,0.006706,0.005882,2016-09-30,1,677.0,39.892557,11.908661,2.717984,13.908509


In [58]:
full_data = full_data[
    full_data.hurricane_hit == 1
].dropna()
print(full_data.shape)
full_data.head()

(1708, 23)


,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit,street_address,city,state,zip,occupancy,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
0,3Q16,2016,"[10] grayson place, 1600 pullen rd, tallahasse...",68.333333,80.0,1.0,733.0,3,1,1600 pullen rd,Tallahassee,FL,32303,0.8929,0.006711,0.005882,2016-09-30,0,748.0,39.482005,11.908661,2.717984,13.908509
1,3Q16,2016,"[10] sabal court, 2125 jackson bluff rd, talla...",68.333333,80.0,1.0,655.0,3,1,2125 jackson bluff rd,Tallahassee,FL,32304,0.9453,0.003298,0.005882,2016-09-30,0,690.0,34.844183,11.908661,2.717984,13.908509
2,3Q16,2016,"[11] capital place at southwood, 2300 bluff oa...",68.333333,80.0,1.0,1233.0,3,1,2300 bluff oak way,Tallahassee,FL,32311,0.9444,0.002880,0.005882,2016-09-30,0,1241.0,30.685704,11.908661,2.717984,13.908509
3,3Q16,2016,"[11] live oaks at 275, 275 john knox rd, talla...",68.333333,80.0,1.0,786.0,3,1,275 john knox rd,Tallahassee,FL,32303,0.9009,0.000797,0.005882,2016-09-30,1,773.0,37.904028,11.908661,2.717984,13.908509
4,3Q16,2016,"[12] capital ridge, 3255 capital cir ne, talla...",68.333333,80.0,1.0,707.0,3,1,3255 capital cir ne,Tallahassee,FL,32308,0.8889,0.006706,0.005882,2016-09-30,1,677.0,39.892557,11.908661,2.717984,13.908509


In [59]:
full_data.groupby('address')['quarter'].agg(list).reset_index().iloc[692].address

'[27] parc500, 500 n congress ave, west palm beach, fl, 33401'

In [60]:
full_data[full_data.address == '[27] parc500, 500 n congress ave, west palm beach, fl, 33401'].drop_duplicates()

,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit,street_address,city,state,zip,occupancy,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
1540,4Q16,2016,"[27] parc500, 500 n congress ave, west palm be...",112.333333,135.0,3.333333,1193.0,1,1,500 n congress ave,West Palm Beach,FL,33401,0.9032,0.007983,0.005005,2016-12-31,0,1342.0,11.2338,38.927992,28.332581,70.963202


## Checkpoint 2 - Data Ready for Modelling [---UPDATE FILE NAME---]

In [61]:
###########################################
######### Update the file name  ###########
###########################################
str_date = str(datetime.now())
full_data.to_parquet(f'../processed_data/LIMITED_full_data_2010_2019_{str_date}.parquet')

In [62]:
f'../processed_data/LIMITED_full_data_2010_2019_{str_date}.parquet'

'../processed_data/LIMITED_full_data_2010_2019_2023-03-28 00:54:10.918250.parquet'

In [63]:
###########################################
######### Update the file name  ###########
###########################################

full_data = pd.read_parquet(f'../processed_data/LIMITED_full_data_2010_2019_{str_date}.parquet')
full_data.head()

,quarter,Year,address,USA_WIND,USA_GUST,USA_SSHS,effective_rent,expose_status_code,hurricane_hit,street_address,city,state,zip,occupancy,prop_rent_growth,sma_rent_growth,quarter_period,rent_decreased,next_efective_rent,distance_from_cost,RISK_SCORE,CFLD_RISKS,HRCN_RISKS
0,3Q16,2016,"[10] grayson place, 1600 pullen rd, tallahasse...",68.333333,80.0,1.0,733.0,3,1,1600 pullen rd,Tallahassee,FL,32303,0.8929,0.006711,0.005882,2016-09-30,0,748.0,39.482005,11.908661,2.717984,13.908509
1,3Q16,2016,"[10] sabal court, 2125 jackson bluff rd, talla...",68.333333,80.0,1.0,655.0,3,1,2125 jackson bluff rd,Tallahassee,FL,32304,0.9453,0.003298,0.005882,2016-09-30,0,690.0,34.844183,11.908661,2.717984,13.908509
2,3Q16,2016,"[11] capital place at southwood, 2300 bluff oa...",68.333333,80.0,1.0,1233.0,3,1,2300 bluff oak way,Tallahassee,FL,32311,0.9444,0.002880,0.005882,2016-09-30,0,1241.0,30.685704,11.908661,2.717984,13.908509
3,3Q16,2016,"[11] live oaks at 275, 275 john knox rd, talla...",68.333333,80.0,1.0,786.0,3,1,275 john knox rd,Tallahassee,FL,32303,0.9009,0.000797,0.005882,2016-09-30,1,773.0,37.904028,11.908661,2.717984,13.908509
4,3Q16,2016,"[12] capital ridge, 3255 capital cir ne, talla...",68.333333,80.0,1.0,707.0,3,1,3255 capital cir ne,Tallahassee,FL,32308,0.8889,0.006706,0.005882,2016-09-30,1,677.0,39.892557,11.908661,2.717984,13.908509


In [64]:
full_data.columns

Index(['quarter', 'Year', 'address', 'USA_WIND', 'USA_GUST', 'USA_SSHS',
       'effective_rent', 'expose_status_code', 'hurricane_hit',
       'street_address', 'city', 'state', 'zip', 'occupancy',
       'prop_rent_growth', 'sma_rent_growth', 'quarter_period',
       'rent_decreased', 'next_efective_rent', 'distance_from_cost',
       'RISK_SCORE', 'CFLD_RISKS', 'HRCN_RISKS'],
      dtype='object')